In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# Imports
# import numpy as np
import argparse
import h5py
import numpy as np
import sys, os
from PIL import Image
import tensorflow as tf

In [3]:
# To allow for below imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# import inception.slim as slim
from inception.slim.scopes import arg_scope
from inception.slim.ops import conv2d, max_pool, fc, flatten, dropout
from inception.slim.losses import cross_entropy_loss

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

FLAGS = None

In [6]:
def coarse_stack(features):
    """Model function for coarse stack."""
    # Input Layer
#     input_layer = tf.reshape(features, [-1, 240, 320, 3])
#     input_layer = tf.reshape(features, [-1, 228, 304, 3])
    input_layer = features
#     print("input_layer: ", input_layer.get_shape())
    tf.summary.image('input', input_layer, 3)

    with arg_scope([conv2d, fc], stddev=0.01,
                   weight_decay=0.0005, activation=tf.nn.relu):
        # See https://github.com/tensorflow/models/blob/master/inception/
        # inception/slim/ops.py for more details.
        #   net = conv2d(inputs=[batch_size, height, width, channels],
        #       num_filters_out, kernel_size=[kernel_height, kernel_width],
        #       stride=int|[stride_height, stride_width],
        #       padding='VALID'|'SAME',
        #       scope='conv1')
        #   net = max_pool(inputs,
        #                           kernel_size=[kernel_height, kernel_width],
        #                           stride=int|[stride_height, stride_width],
        #                           scope='pool1')

        net = conv2d(input_layer, num_filters_out=96, kernel_size=[11, 11],
                     stride=4, padding='VALID', scope='conv1')
#         print("conv1: ", net.get_shape())

        net = max_pool(net, kernel_size=[3, 3], stride=[2, 2],
                       padding='SAME', scope='pool1')
#         print("pool1: ", net.get_shape())

        net = conv2d(net, num_filters_out=256, kernel_size=[5, 5], stride=1,
                     padding='SAME', scope='conv2')
#         print("conv2: ", net.get_shape())

        net = max_pool(net, kernel_size=[3, 3], stride=[2, 2],
                       padding='SAME', scope='pool2')
#         print("pool2: ", net.get_shape())

        net = conv2d(net, num_filters_out=384, kernel_size=[3, 3], stride=1,
                     padding='SAME', scope='conv3')
#         print("conv3: ", net.get_shape())

        net = conv2d(net, num_filters_out=384, kernel_size=[3, 3], stride=1,
                     padding='SAME', scope='conv4')
#         print("conv4: ", net.get_shape())

        net = conv2d(net, num_filters_out=256, kernel_size=[3, 3], stride=1,
                     padding='SAME', scope='conv5')
#         print("conv5: ", net.get_shape())

        net = max_pool(net, kernel_size=[3, 3], stride=[2, 2],
                       scope='pool5')
#         print("pool5: ", net.get_shape())

        net = flatten(net, scope='flatten5')
#         print("flatten5: ", net.get_shape())
        
        net = fc(net, 4096, scope='full1')
#         print("full1: ", net.get_shape())
        
        net = dropout(net, keep_prob=0.5, scope='dropout_full1')
#         print("dropout_full1: ", net.get_shape())
        
        net = fc(net, 4070, scope='full2')
#         print("full2: ", net.get_shape())
        
        net = dropout(net, 0.5, scope='dropout_full2')
#         print("dropout_full2: ", net.get_shape())
        
        net = tf.reshape(net, shape=[-1, 74, 55, 1])
#         print("reshape: ", net.get_shape())
        
        tf.summary.tensor_summary('inception_net', net)
        
        return net

In [7]:
def fine_stack(features, coarse_pred):
    """Model function for coarse stack."""
    # Input Layer
#     input_layer = tf.reshape(features, [-1, 240, 320, 3])
#     input_layer = tf.reshape(features, [-1, 228, 304, 3])
    input_layer = features
#     print("input_layer: ", input_layer.get_shape())

    with arg_scope([conv2d, fc], stddev=0.01,
                   weight_decay=0.0005, activation=tf.nn.relu):
        net = conv2d(input_layer, num_filters_out=63, kernel_size=[9, 9],
                     stride=2, padding='VALID', scope='fine_conv1')
#         print("fine_conv1: ", net.get_shape())

        net = max_pool(net, kernel_size=[3, 3], stride=[2, 2],
                       padding='SAME', scope='fine_pool1')
#         print("fine_pool1: ", net.get_shape())
        
        # concat coarse predictioins with fine_pool1 along channel axis
        net = tf.concat([net, coarse_pred], 3)
#         print("concat: ", net.get_shape())
        
        net = conv2d(net, num_filters_out=64, kernel_size=[5, 5],
                     stride=1, padding='SAME', scope='fine_conv2')
#         print("fine_conv2: ", net.get_shape())
        
        net = conv2d(net, num_filters_out=1, kernel_size=[5, 5],
                     activation=None, stride=1, padding='SAME', scope='fine_conv3') # Linear read-out layer
#         print("fine_conv3: ", net.get_shape())
        
        tf.summary.tensor_summary('fine_net', net)
        tf.summary.image('output', net, 3)
        
        return net

In [8]:
def get_data():
    f = h5py.File('/home/rayner/Downloads/nyu_depth_v2_labeled.mat')
    images = f['images'] # images.shape = (1449, 3, 640, 480)
    images = np.transpose(images, axes=(0, 2, 3, 1)) # images.shape = (1449, 640, 480, 3)
    depths = f['depths'] # depths.shape = (1449, 640, 480)
    del(f) # free up memory
    # downsize images and depths
    images = [Image.fromarray(image, mode='RGB').resize(size=(228, 304), resample=Image.BICUBIC) for image in images]
    depths = [Image.fromarray(image).resize(size=(55, 74), resample=Image.BICUBIC) for image in depths]
    # convert back to nparrays
    images = [np.asarray(image) for image in images]
    images = np.array(images) # ndarray
    depths = [np.asarray(image) for image in depths]
    depths = np.expand_dims(depths, axis=3) # depths.shape = (1449, 640, 480, 1) an ndarray
    return images, depths

In [9]:
def train():
    # Get training data
    images, depths = get_data()
    N = len(images)
    shuffler = np.random.permutation(N) # shuffle dataset
    Ntest = int(0.2* N)
    Ntrain = int(0.8 * (N-Ntest))
    Nvalid = int(0.2 * (N-Ntest))
    
    print(Ntrain, Ntest, Nvalid)
    
    x_train = images[shuffler[:Ntrain]]
    y_train = depths[shuffler[:Ntrain]]
    
    x_valid = images[shuffler[Ntrain:Ntrain+Nvalid]]
    y_valid = depths[shuffler[Ntrain:Ntrain+Nvalid]]
    
    x_test = images[shuffler[-Ntest:]]
    y_test = depths[shuffler[-Ntest:]]

    x = tf.placeholder(tf.float32, shape=[None, 304, 228, 3])
    y_ = tf.placeholder(tf.float32, shape=[None, 74, 55, 1])
    coarse_model = coarse_stack(x)
    y = fine_stack(x, coarse_model) # shape fine_conv3:  (?, 74, 55, 1)
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        # define loss
        loss = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=y))
        print("loss: ", loss)
        tf.summary.scalar('mean_squared_error', loss)
        train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)
        
        # Merge all summaries
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('/tmp/tensorflow/eigen/logs/init_model' + '/train', sess.graph)
        test_writer = tf.summary.FileWriter('/tmp/tensorflow/eigen/logs/init_model' + '/test')
        
        # Add an op to initialize the variables.
        sess.run(tf.global_variables_initializer())
        
        batch_size = 50
        batch_start = 0
        
        for i in range(1000):
#             batch = mnist.train.next_batch(50)
            if batch_start + batch_size < Ntrain:
                batch_end = batch_start + batch_size
            else:
                batch_end = Ntrain
            
#             x_feed = np.asarray(x_train[batch_start:batch_end])
#             y_feed = np.asarray(y_train[batch_start:batch_end])
            x_feed = x_train[batch_start:batch_end]
            y_feed = y_train[batch_start:batch_end]
            if i % 10 == 0: # Record summaries and valid-set accuracy
                # train_accuracy = accuracy.eval(feed_dict={
                #     x: batch[0], y_: batch[1]
                #     })
                summary = sess.run(merged, feed_dict={
                    x: x_valid, y_: y_valid
                })
                test_writer.add_summary(summary, i)
                print("Valid loss at step %s: %s" %(i, loss))
            else: # Record train set summaries and train
                
#                 print("%s: start = %s, end = %s" %(i, batch_start, batch_end))
                if i % 100 == 99: # Record execution stats
                    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    run_metadata = tf.RunMetadata()
                    summary, _ = sess.run([merged, train_step],
                        options = run_options,
                        run_metadata = run_metadata,
                        feed_dict={
                            x: x_feed, 
                            y_: y_feed
                            }
                        )
                    train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                    train_writer.add_summary(summary, i)
                else: # Record a summary
                    summary, _ = sess.run([merged, train_step],
                        feed_dict={
                            x: x_feed, 
                            y_: y_feed
                            }
                        )
                    train_writer.add_summary(summary, i)

            batch_start = batch_end % Ntrain
            # Save the variables to disk.
            save_path = saver.save(sess, "/tmp/tensorflow/eigen/init_model.ckpt")
            print("Model saved in file: %s" % save_path)
                      
        summary = sess.run(merged, feed_dict={
                    x: x_test, y_: y_test
                })
#         test_writer.add_summary(summary, i)
        print("Test error: %s" %(loss))
                      
        train_writer.close()
        test_writer.close()
        
#         # Save the variables to disk.
#         save_path = saver.save(sess, "/tmp/tensorflow/eigen/init_model.ckpt")
#         print("Model saved in file: %s" % save_path)

In [10]:
# Main
if tf.gfile.Exists('/tmp/tensorflow/eigen/logs/init_model'):
        tf.gfile.DeleteRecursively('/tmp/tensorflow/eigen/logs/init_model')
tf.gfile.MakeDirs('/tmp/tensorflow/eigen/logs/init_model')
train()

928 289 232
loss:  Tensor("Mean:0", shape=(), dtype=float32)
Valid loss at step 0: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 10: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 130: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_m

Valid loss at step 250: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 260: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 380: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 390: Tensor("Mean:0", shape=()

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 510: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_m

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 630: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 640: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 760: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_m

Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 880: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Valid loss at step 890: Tensor("Mean:0", shape=(), dtype=float32)
Model saved in file: /tmp/tensorflow/eigen/init_model.ckpt
Model saved in file: /tmp/tensorflow/eigen

In [ ]:
def predict(image):
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    # Later, launch the model, use the saver to restore variables from disk, and
    # do some work with the model.
    with tf.Session() as sess:
        # Restore variables from disk.
        saver.restore(sess, "/tmp/tensorflow/eigen/init_model.ckpt")
        print("Model restored.")
        # Do some work with the model
        print(sess)

In [ ]:
images, depths = get_data()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(images[0])
plt.show()
plt.imshow(depths[0][:,:,-1])
plt.show()

In [ ]:
predict(image=images[0])